### Add downstream PFAFIDs to merged shapefile

* Purpose of script: Create a csv file with all PFAFID's downstream
* Author: Rutger Hofste
* Kernel used: python27
* Date created: 20170823

In [1]:
import pandas as pd
import numpy as np
import os, ftplib
from datetime import datetime, timedelta
from ast import literal_eval

In [2]:
S3_INPUT_PATH = "s3://wri-projects/Aqueduct30/processData/Y2017M08D22_RH_Upstream_V01/output/"
S3_OUTPUT_PATH = "s3://wri-projects/Aqueduct30/processData/Y2017M08D23_RH_Downstream_V01/output/"
EC2_INPUT_PATH = "/volumes/data/Y2017M08D23_RH_Downstream_V01/input/"
EC2_OUTPUT_PATH = "/volumes/data/Y2017M08D23_RH_Downstream_V01/output/"
INPUT_FILENAME = "hybas_lev06_v1c_merged_fiona_upstream_V01.csv"
OUTPUT_FILENAME = "hybas_lev06_v1c_merged_fiona_upstream_downstream_V01.csv"

In [3]:
!mkdir -p {EC2_INPUT_PATH}
!mkdir -p {EC2_OUTPUT_PATH}

In [4]:
!aws s3 cp {S3_INPUT_PATH} {EC2_INPUT_PATH} --recursive

download: s3://wri-projects/Aqueduct30/processData/Y2017M08D22_RH_Upstream_V01/output/hybas_lev06_v1c_merged_fiona_upstream_V01.csv to ../../../../data/Y2017M08D23_RH_Downstream_V01/input/hybas_lev06_v1c_merged_fiona_upstream_V01.csv
download: s3://wri-projects/Aqueduct30/processData/Y2017M08D22_RH_Upstream_V01/output/hybas_lev06_v1c_merged_fiona_V01.csv to ../../../../data/Y2017M08D23_RH_Downstream_V01/input/hybas_lev06_v1c_merged_fiona_V01.csv


In [5]:
def stringToList(string):
    # input format : "[42, 42, 42]" , note the spaces after the commas, in this case I have a list of integers
    string = string[1:len(string)-1]
    try:
        if len(string) != 0: 
            tempList = string.split(", ")
            newList = list(map(lambda x: int(x), tempList))
        else:
            newList = []
    except:
        newList = [-9999]
    return(newList)

In [9]:
inputLocation = os.path.join(EC2_INPUT_PATH,INPUT_FILENAME)
outputLocation = os.path.join(EC2_OUTPUT_PATH,OUTPUT_FILENAME)

In [10]:
df = pd.read_csv(inputLocation)

In [11]:
df.head()

,Unnamed: 0,HYBAS_ID,NEXT_DOWN,NEXT_SINK,MAIN_BAS,DIST_SINK,DIST_MAIN,SUB_AREA,UP_AREA,PFAF_ID,ENDO,COAST,ORDER,SORT,Upstream_HYBAS_IDs,Upstream_PFAF_IDs
0,0,6060000010,0,6060000010,6060000010,0.0,0.0,4317.4,4317.4,611001,0,1,0,1,[],[]
1,1,6060000200,0,6060000200,6060000200,0.0,0.0,35995.5,35996.7,611002,0,0,1,2,[],[]
2,2,6060000210,0,6060000210,6060000210,0.0,0.0,443.9,443.9,611003,0,1,0,3,[],[]
3,3,6060000240,0,6060000240,6060000240,0.0,0.0,2186.3,2186.3,611004,0,0,1,4,[],[]
4,4,6060000250,0,6060000250,6060000250,0.0,0.0,6533.8,6533.8,611005,0,1,0,5,[],[]


In [12]:
df["Upstream_HYBAS_IDs"] = df["Upstream_HYBAS_IDs"].apply(lambda x: stringToList(x))
df["Upstream_PFAF_IDs"] = df["Upstream_PFAF_IDs"].apply(lambda x: stringToList(x))

In [13]:
header = df.dtypes

df["HYBAS_ID2"] = df["HYBAS_ID"]
df = df.set_index(["HYBAS_ID2"])


# testing purposes
#df = df[ 0:200]
df_out = df.copy()


df_out['Downstream_HYBAS_IDs'] = "Nodata"
df_out['Downstream_PFAF_IDs'] = "Nodata"

print(df_out.dtypes)

i = 1
for id in df.index:
    print("item: ", i, " id: ",id)
    i += 1
    writeID = id
    allDownID = []
    allDownPFAF = []    

    sinkHybasID = np.int64(df.loc[id]["NEXT_SINK"])
    sinkPfafID = np.int64(df.loc[sinkHybasID]["PFAF_ID"])


    while id != 0:
              
        series = df.loc[id]
        downId = np.int64(series["NEXT_DOWN"]) # Next down ID
        if downId != 0 :
            downSeries = df.loc[downId]
            pfafID = np.int64(downSeries["PFAF_ID"])
            allDownID.append(downId)
            allDownPFAF.append(pfafID)
            id = downId
        else:
            id = 0
            pass

    df_out.set_value(writeID, 'Downstream_HYBAS_IDs', allDownID)
    df_out.set_value(writeID, 'Downstream_PFAF_IDs',allDownPFAF)

    df_out.set_value(writeID, 'NEXT_SINK_PFAF', sinkPfafID )


df_out.to_csv(outputLocation)

print("done")

Unnamed: 0                int64
HYBAS_ID                  int64
NEXT_DOWN                 int64
NEXT_SINK                 int64
MAIN_BAS                  int64
DIST_SINK               float64
DIST_MAIN               float64
SUB_AREA                float64
UP_AREA                 float64
PFAF_ID                   int64
ENDO                      int64
COAST                     int64
ORDER                     int64
SORT                      int64
Upstream_HYBAS_IDs       object
Upstream_PFAF_IDs        object
Downstream_HYBAS_IDs     object
Downstream_PFAF_IDs      object
dtype: object
item:  1  id:  6060000010
item:  2  id:  6060000200
item:  3  id:  6060000210
item:  4  id:  6060000240
item:  5  id:  6060000250
item:  6  id:  6060000450
item:  7  id:  6060000460
item:  8  id:  6060000580
item:  9  id:  6060000590
item:  10  id:  6060000740
item:  11  id:  6060068100
item:  12  id:  6060068110
item:  13  id:  6060070260
item:  14  id:  6060070270
item:  15  id:  6060070390
item:  16  id:

item:  303  id:  6060267790
item:  304  id:  6060263530
item:  305  id:  6060263540
item:  306  id:  6060262220
item:  307  id:  6060262110
item:  308  id:  6060194730
item:  309  id:  6060194720
item:  310  id:  6060262200
item:  311  id:  6060261920
item:  312  id:  6060277380
item:  313  id:  6060277480
item:  314  id:  6060197930
item:  315  id:  6060198030
item:  316  id:  6060298170
item:  317  id:  6060298270
item:  318  id:  6060290960
item:  319  id:  6060291060
item:  320  id:  6060250260
item:  321  id:  6060250180
item:  322  id:  6060295230
item:  323  id:  6060295220
item:  324  id:  6060182970
item:  325  id:  6060182980
item:  326  id:  6060246620
item:  327  id:  6060246540
item:  328  id:  6060242270
item:  329  id:  6060242260
item:  330  id:  6060295570
item:  331  id:  6060295580
item:  332  id:  6060188720
item:  333  id:  6060188710
item:  334  id:  6060317490
item:  335  id:  6060317690
item:  336  id:  6060266340
item:  337  id:  6060266470
item:  338  id:  606

item:  598  id:  6060204620
item:  599  id:  6060204430
item:  600  id:  6060471640
item:  601  id:  6060471650
item:  602  id:  6060190800
item:  603  id:  6060190720
item:  604  id:  6060243550
item:  605  id:  6060243660
item:  606  id:  6060279130
item:  607  id:  6060279120
item:  608  id:  6060206300
item:  609  id:  6060206500
item:  610  id:  6060491730
item:  611  id:  6060491720
item:  612  id:  6060493160
item:  613  id:  6060493260
item:  614  id:  6060494510
item:  615  id:  6060494500
item:  616  id:  6060504910
item:  617  id:  6060504920
item:  618  id:  6060238540
item:  619  id:  6060238550
item:  620  id:  6060171200
item:  621  id:  6060171210
item:  622  id:  6060502630
item:  623  id:  6060502760
item:  624  id:  6060196210
item:  625  id:  6060196330
item:  626  id:  6060525230
item:  627  id:  6060525240
item:  628  id:  6060507380
item:  629  id:  6060507240
item:  630  id:  6060524030
item:  631  id:  6060524130
item:  632  id:  6060404060
item:  633  id:  606

item:  949  id:  6060435010
item:  950  id:  6060435170
item:  951  id:  6060477660
item:  952  id:  6060477510
item:  953  id:  6060460890
item:  954  id:  6060461030
item:  955  id:  6060466410
item:  956  id:  6060466420
item:  957  id:  6060009770
item:  958  id:  6060010100
item:  959  id:  6060010110
item:  960  id:  6060010170
item:  961  id:  6060010180
item:  962  id:  6060010250
item:  963  id:  6060010260
item:  964  id:  6060010440
item:  965  id:  6060010450
item:  966  id:  6060010480
item:  967  id:  6060010490
item:  968  id:  6060010510
item:  969  id:  6060338070
item:  970  id:  6060337870
item:  971  id:  6060373290
item:  972  id:  6060373400
item:  973  id:  6060380590
item:  974  id:  6060380720
item:  975  id:  6060381820
item:  976  id:  6060382080
item:  977  id:  6060010520
item:  978  id:  6060010670
item:  979  id:  6060370750
item:  980  id:  6060370930
item:  981  id:  6060378220
item:  982  id:  6060378380
item:  983  id:  6060386940
item:  984  id:  606

item:  1259  id:  6060853440
item:  1260  id:  6060811150
item:  1261  id:  6060811230
item:  1262  id:  6060818720
item:  1263  id:  6060818840
item:  1264  id:  6060016770
item:  1265  id:  6060016970
item:  1266  id:  6060893010
item:  1267  id:  6060892920
item:  1268  id:  6060887830
item:  1269  id:  6060887840
item:  1270  id:  6060887140
item:  1271  id:  6060887050
item:  1272  id:  6060879900
item:  1273  id:  6060879890
item:  1274  id:  6060878610
item:  1275  id:  6060878570
item:  1276  id:  6060870380
item:  1277  id:  6060870370
item:  1278  id:  6060869680
item:  1279  id:  6060869670
item:  1280  id:  6060884270
item:  1281  id:  6060884280
item:  1282  id:  6060883820
item:  1283  id:  6060883900
item:  1284  id:  6060849780
item:  1285  id:  6060849790
item:  1286  id:  6060846530
item:  1287  id:  6060846520
item:  1288  id:  6060837900
item:  1289  id:  6060837910
item:  1290  id:  6060837570
item:  1291  id:  6060837640
item:  1292  id:  6060876090
item:  1293  i

...

item:  15139  id:  3060511230
item:  15140  id:  3060660640
item:  15141  id:  3060660730
item:  15142  id:  3060637090
item:  15143  id:  3060637200
item:  15144  id:  3060557960
item:  15145  id:  3060558130
item:  15146  id:  3060651650
item:  15147  id:  3060651560
item:  15148  id:  3060635420
item:  15149  id:  3060635570
item:  15150  id:  3060650360
item:  15151  id:  3060650460
item:  15152  id:  3060550600
item:  15153  id:  3060550480
item:  15154  id:  3060671720
item:  15155  id:  3060671820
item:  15156  id:  3060680570
item:  15157  id:  3060680690
item:  15158  id:  3060686240
item:  15159  id:  3060686230
item:  15160  id:  3060677350
item:  15161  id:  3060677450
item:  15162  id:  3060679680
item:  15163  id:  3060679820
item:  15164  id:  3060699480
item:  15165  id:  3060699570
item:  15166  id:  3060705860
item:  15167  id:  3060705700
item:  15168  id:  3060693220
item:  15169  id:  3060693280
item:  15170  id:  3060709620
item:  15171  id:  3060709670
item:  151

item:  15414  id:  3060495970
item:  15415  id:  3060633280
item:  15416  id:  3060633290
item:  15417  id:  3060606360
item:  15418  id:  3060606730
item:  15419  id:  3060639100
item:  15420  id:  3060639150
item:  15421  id:  3060642770
item:  15422  id:  3060642780
item:  15423  id:  3060650910
item:  15424  id:  3060650900
item:  15425  id:  3060414840
item:  15426  id:  3060414830
item:  15427  id:  3060545280
item:  15428  id:  3060545290
item:  15429  id:  3060558830
item:  15430  id:  3060558930
item:  15431  id:  3060654760
item:  15432  id:  3060654610
item:  15433  id:  3060651040
item:  15434  id:  3060650920
item:  15435  id:  3060448050
item:  15436  id:  3060448060
item:  15437  id:  3060658430
item:  15438  id:  3060658420
item:  15439  id:  3060630700
item:  15440  id:  3060630590
item:  15441  id:  3060596090
item:  15442  id:  3060596100
item:  15443  id:  3060659790
item:  15444  id:  3060659700
item:  15445  id:  3060629440
item:  15446  id:  3060629610
item:  154

item:  15723  id:  3060258480
item:  15724  id:  3060258220
item:  15725  id:  3060264560
item:  15726  id:  3060264710
item:  15727  id:  3060268880
item:  15728  id:  3060269050
item:  15729  id:  3060282160
item:  15730  id:  3060282370
item:  15731  id:  3060297840
item:  15732  id:  3060297700
item:  15733  id:  3060297180
item:  15734  id:  3060297030
item:  15735  id:  3060299300
item:  15736  id:  3060299510
item:  15737  id:  3060299520
item:  15738  id:  3060299320
item:  15739  id:  3060303860
item:  15740  id:  3060303460
item:  15741  id:  3060287750
item:  15742  id:  3060287680
item:  15743  id:  3060290270
item:  15744  id:  3060290500
item:  15745  id:  3060293050
item:  15746  id:  3060292920
item:  15747  id:  3060269350
item:  15748  id:  3060269230
item:  15749  id:  3060295490
item:  15750  id:  3060295330
item:  15751  id:  3060295480
item:  15752  id:  3060295620
item:  15753  id:  3060350110
item:  15754  id:  3060350280
item:  15755  id:  3060298910
item:  157

item:  16039  id:  3060224120
item:  16040  id:  3060009680
item:  16041  id:  3060009770
item:  16042  id:  3060009780
item:  16043  id:  3060010130
item:  16044  id:  3060010140
item:  16045  id:  3060010150
item:  16046  id:  3060071570
item:  16047  id:  3060071560
item:  16048  id:  3060076900
item:  16049  id:  3060076970
item:  16050  id:  3060080980
item:  16051  id:  3060080740
item:  16052  id:  3060084910
item:  16053  id:  3060084900
item:  16054  id:  3060010160
item:  16055  id:  3060010310
item:  16056  id:  3060010320
item:  16057  id:  3060010470
item:  16058  id:  3060078360
item:  16059  id:  3060078520
item:  16060  id:  3060084760
item:  16061  id:  3060084820
item:  16062  id:  3060104900
item:  16063  id:  3060104890
item:  16064  id:  3060117600
item:  16065  id:  3060117610
item:  16066  id:  3060136060
item:  16067  id:  3060136270
item:  16068  id:  3060134090
item:  16069  id:  3060134080
item:  16070  id:  3060134710
item:  16071  id:  3060134760
item:  160

item:  16355  id:  3060024380
item:  16356  id:  3060024390
item:  16357  id:  3060024400
item:  16358  id:  3060024410
item:  16359  id:  3060024420
item:  16360  id:  3060024430
item:  16361  id:  3060024440
item:  16362  id:  3060024450
item:  16363  id:  3060024460
item:  16364  id:  3060024470
item:  16365  id:  3060024480
item:  16366  id:  3060024490
item:  16367  id:  3060024510
item:  16368  id:  3060024530
item:  16369  id:  3060024540
item:  16370  id:  3060024570
item:  16371  id:  3060024580
item:  16372  id:  3060024610
item:  16373  id:  3060024620
item:  16374  id:  3060024640
item:  16375  id:  3060024670
item:  16376  id:  3060024690
item:  16377  id:  3060024710
item:  16378  id:  3060024780
item:  16379  id:  3060024830
item:  16380  id:  3060024860
item:  16381  id:  3060024870
item:  16382  id:  3060024890
item:  16383  id:  3060024960
item:  16384  id:  3060024970
item:  16385  id:  3060025010
item:  16386  id:  3060025070
item:  16387  id:  3060025110
item:  163

In [14]:
df_out.tail()

,Unnamed: 0,HYBAS_ID,NEXT_DOWN,NEXT_SINK,MAIN_BAS,DIST_SINK,DIST_MAIN,SUB_AREA,UP_AREA,PFAF_ID,ENDO,COAST,ORDER,SORT,Upstream_HYBAS_IDs,Upstream_PFAF_IDs,Downstream_HYBAS_IDs,Downstream_PFAF_IDs,NEXT_SINK_PFAF
HYBAS_ID2,,,,,,,,,,,,,,,,,,,
3060025460,16392,3060025460,3060001840,3060025460,3060001840,0.0,1.0,2489.4,2489.4,312042,2,0,2,1502,[],[],[3060001840],[312101],312042.0
3060025570,16393,3060025570,3060001840,3060025570,3060001840,0.0,1.0,5471.0,5471.0,312016,2,0,2,1503,[],[],[3060001840],[312101],312016.0
3060026330,16394,3060026330,3060496590,3060026330,3060001840,0.0,2229.1,539.7,539.9,312804,2,0,3,1504,[],[],"[3060496590, 3060490890, 3060486480, 306048696...","[312811, 312709, 312707, 312705, 312703, 31270...",312804.0
3060026650,16395,3060026650,3060607050,3060026650,3060004740,0.0,2549.4,445.5,445.6,322660,2,0,4,1505,[],[],"[3060607050, 3060601680, 3060593200, 306057269...","[322661, 322650, 322639, 322637, 322635, 32263...",322660.0
3060027750,16396,3060027750,3060530840,3060027750,3060001840,0.0,2245.4,480.7,480.7,312670,2,0,3,1506,[],[],"[3060530840, 3060511890, 3060493080, 306048193...","[312671, 312650, 312630, 312610, 312590, 31257...",312670.0


In [16]:
df_out.tail()

,Unnamed: 0,HYBAS_ID,NEXT_DOWN,NEXT_SINK,MAIN_BAS,DIST_SINK,DIST_MAIN,SUB_AREA,UP_AREA,PFAF_ID,ENDO,COAST,ORDER,SORT,Upstream_HYBAS_IDs,Upstream_PFAF_IDs,Downstream_HYBAS_IDs,Downstream_PFAF_IDs,NEXT_SINK_PFAF,Basin_HYBAS_IDs
HYBAS_ID2,,,,,,,,,,,,,,,,,,,,
3060025460,16392,3060025460,3060001840,3060025460,3060001840,0.0,1.0,2489.4,2489.4,312042,2,0,2,1502,[],[],[3060001840],[312101],312042.0,[3060001840]
3060025570,16393,3060025570,3060001840,3060025570,3060001840,0.0,1.0,5471.0,5471.0,312016,2,0,2,1503,[],[],[3060001840],[312101],312016.0,[3060001840]
3060026330,16394,3060026330,3060496590,3060026330,3060001840,0.0,2229.1,539.7,539.9,312804,2,0,3,1504,[],[],"[3060496590, 3060490890, 3060486480, 306048696...","[312811, 312709, 312707, 312705, 312703, 31270...",312804.0,"[3060496590, 3060490890, 3060486480, 306048696..."
3060026650,16395,3060026650,3060607050,3060026650,3060004740,0.0,2549.4,445.5,445.6,322660,2,0,4,1505,[],[],"[3060607050, 3060601680, 3060593200, 306057269...","[322661, 322650, 322639, 322637, 322635, 32263...",322660.0,"[3060607050, 3060601680, 3060593200, 306057269..."
3060027750,16396,3060027750,3060530840,3060027750,3060001840,0.0,2245.4,480.7,480.7,312670,2,0,3,1506,[],[],"[3060530840, 3060511890, 3060493080, 306048193...","[312671, 312650, 312630, 312610, 312590, 31257...",312670.0,"[3060530840, 3060511890, 3060493080, 306048193..."


In [44]:
def rutger(row):
    return row["Downstream_HYBAS_IDs"].append(row["Upstream_HYBAS_IDs"])

In [45]:
df_out['Basin_HYBAS_IDs'] = df_out.apply(rutger, axis=1)

In [46]:
df_out.head()

,Unnamed: 0,HYBAS_ID,NEXT_DOWN,NEXT_SINK,MAIN_BAS,DIST_SINK,DIST_MAIN,SUB_AREA,UP_AREA,PFAF_ID,ENDO,COAST,ORDER,SORT,Upstream_HYBAS_IDs,Upstream_PFAF_IDs,Downstream_HYBAS_IDs,Downstream_PFAF_IDs,NEXT_SINK_PFAF,Basin_HYBAS_IDs
HYBAS_ID2,,,,,,,,,,,,,,,,,,,,
6060000010,0,6060000010,0,6060000010,6060000010,0.0,0.0,4317.4,4317.4,611001,0,1,0,1,[],[],[[]],[],611001.0,None
6060000200,1,6060000200,0,6060000200,6060000200,0.0,0.0,35995.5,35996.7,611002,0,0,1,2,[],[],[[]],[],611002.0,None
6060000210,2,6060000210,0,6060000210,6060000210,0.0,0.0,443.9,443.9,611003,0,1,0,3,[],[],[[]],[],611003.0,None
6060000240,3,6060000240,0,6060000240,6060000240,0.0,0.0,2186.3,2186.3,611004,0,0,1,4,[],[],[[]],[],611004.0,None
6060000250,4,6060000250,0,6060000250,6060000250,0.0,0.0,6533.8,6533.8,611005,0,1,0,5,[],[],[[]],[],611005.0,None


In [17]:
test = df_out.loc[6060068110]

In [18]:
print(test)

Unnamed: 0                                                             11
HYBAS_ID                                                       6060068110
NEXT_DOWN                                                      6060000740
NEXT_SINK                                                      6060000740
MAIN_BAS                                                       6060000740
DIST_SINK                                                           277.1
DIST_MAIN                                                           277.1
SUB_AREA                                                            610.1
UP_AREA                                                            225688
PFAF_ID                                                            612030
ENDO                                                                    0
COAST                                                                   0
ORDER                                                                   1
SORT                                  

In [30]:
weirdString = list(test.Upstream_PFAF_IDs  + test.Upstream_PFAF_IDs + test.HYBAS_ID)

In [31]:
type(weirdString)

list

In [32]:
print(weirdString)

[6060680151, 6060680160, 6060680153, 6060680152, 6060680180, 6060680170, 6060680154, 6060680155, 6060680190, 6060680201, 6060680203, 6060680202, 6060680205, 6060680204, 6060680157, 6060680156, 6060680159, 6060680158, 6060680206, 6060680207, 6060680209, 6060680208, 6060680151, 6060680160, 6060680153, 6060680152, 6060680180, 6060680170, 6060680154, 6060680155, 6060680190, 6060680201, 6060680203, 6060680202, 6060680205, 6060680204, 6060680157, 6060680156, 6060680159, 6060680158, 6060680206, 6060680207, 6060680209, 6060680208]


In [21]:
!aws s3 cp {EC2_OUTPUT_PATH} {S3_OUTPUT_PATH} --recursive

upload: ../../../../data/Y2017M08D23_RH_Downstream_V01/output/hybas_lev06_v1c_merged_fiona_upstream_downstream_V01.csv to s3://wri-projects/Aqueduct30/processData/Y2017M08D23_RH_Downstream_V01/output/hybas_lev06_v1c_merged_fiona_upstream_downstream_V01.csv
